# 管道提示词

管道提示词可以将多个提示组合在一起。当您想要重复使用部分提示时，这会很有用。这可以通过 PipelinePrompt 来完成。

PipelinePrompt 由两个主要部分组成：
- 最终提示：返回的最终提示
- 管道提示：元组列表，由字符串名称和提示模板组成。每个提示模板将被格式化，然后作为具有相同名称的变量传递到未来的提示模板。

In [1]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts import PromptTemplate

full_template = """\
    {introduction}
    {examples}
    {start}
"""
full_prompt = PromptTemplate.from_template(full_template)

In [2]:
introduction_template = "你在冒充{person}."
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [3]:
examples_template = """下面是一些交互示例\n
Q: {question_q}
A: {question_a}
"""

examples_prompt = PromptTemplate.from_template(examples_template)

In [4]:
start_template = """现在正式开始：\n
Q: {input}
A: 
"""

start_prompt = PromptTemplate.from_template(start_template)

In [5]:
pipeline_prompt = [
    ("introduction", introduction_prompt),
    ("examples", examples_prompt),
    ("start", start_prompt)
]

final_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt,
    pipeline_prompts=pipeline_prompt
)


final_prompt.input_variables

/tmp/ipykernel_12252/2544058237.py:7: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  final_prompt = PipelinePromptTemplate(


['person', 'question_a', 'input', 'question_q']

In [6]:
print(final_prompt.format(
    person = "小明",
    question_q = "你喜欢什么水果？",
    question_a = '西瓜',
    input = '你最喜欢的短视频网站是什么？'
))

    你在冒充小明.
    下面是一些交互示例

Q: 你喜欢什么水果？
A: 西瓜

    现在正式开始：

Q: 你最喜欢的短视频网站是什么？
A: 




In [7]:
# 结合大模型
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192, model="glm-4-plus")

chain = final_prompt | model | StrOutputParser()

for s in chain.stream(
    {
        "person":"小明",
        "question_q": "你喜欢什么水果？",
        "question_a": '西瓜',
        "input": '你最喜欢的短视频网站是什么？'
    }
):
    print(s, end="", flush=True)

抖音

In [8]:
for s in chain.stream(
    {
        "person":"小明",
        "question_q": "你喜欢什么水果？",
        "question_a": '西瓜',
        "input": '为什么不是西瓜视频？'
    }
):
    print(s, end="", flush=True)

因为我更喜欢真的西瓜，吃起来又甜又解渴，而且营养丰富。西瓜视频虽然也不错，但和吃西瓜的感觉还是不一样的嘛！😄